<a href="https://colab.research.google.com/github/spardeshi/AML-service-labs/blob/master/MIMICNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Install Latest Version of Some Packages
!pip install --upgrade chart_studio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 3.2 MB/s eta 0:00:00


In [2]:
#@title Import Python Libraries & Some Other Setup
# Basic Python data science libraries
import pandas as pd
import numpy as np
import scipy.optimize

# Import and setup for plotly in Colab
import chart_studio
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px

# Enable displaying pandas data frames as interactive tables by default
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [3]:
#@title Provide Google Credentials to Colab Runtime (May Require Manually Copy/Pasting Authentication Code)
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [15]:
#@title Enter Google Cloud/BigQuery Project ID
project_id = 'physionet-data-449700' #@param{type:"string"}

# Package used for interfacing w/ BigQuery from Python
from google.cloud import bigquery

# Create BigQuery client
bq_client = bigquery.Client(project = project_id)

In [16]:
datasets = list(bq_client.list_datasets())
if datasets:
    print("Datasets in project:", project_id)
    for dataset in datasets:
        print(f"- {dataset.dataset_id}")
else:
    print("No datasets found in project.")

Datasets in project: physionet-data-449700
- mimiciii_clinical
- mimiciii_notes


In [17]:
dataset_id = f"{project_id}.mimiciii_clinical"
tables = list(bq_client.list_tables(dataset_id))

print(f"Tables in dataset {dataset_id}:")
for table in tables:
    print(f"- {table.table_id}")

Tables in dataset physionet-data-449700.mimiciii_clinical:
- diagnoses_icd


In [18]:
dataset_id_notes = f"{project_id}.mimiciii_notes"
tables = list(bq_client.list_tables(dataset_id_notes))

print(f"Tables in dataset {dataset_id_notes}:")
for table in tables:
    print(f"- {table.table_id}")

Tables in dataset physionet-data-449700.mimiciii_notes:
- noteevents


In [19]:
query_noteevents = """
SELECT subject_id, hadm_id, chartdate, category, description, text
FROM `physionet-data-449700.mimiciii_notes.noteevents`
"""
query_job = bq_client.query(query_noteevents)
df_noteevents = query_job.to_dataframe()

df_noteevents.head()

,subject_id,hadm_id,chartdate,category,description,text
0,59946,107407,2199-01-04,Social Work,Social Work Admission Note,Family Information\n Next of [**Doctor First...
1,21514,140376,2108-09-05,General,ICU Event Note,Clinician: Nurse\n Pt again refuses all car...
2,94906,169117,2106-11-30,Rehab Services,Physical Therapy Progress Note,"Subjective:\n "" I feel a little better today..."
3,52021,175440,2145-01-27,Rehab Services,Physical Therapy Evaluation Note,Attending Physician: [**Name10 (NameIs) 1638**...
4,28973,158953,2159-02-26,Social Work,Social Work Progress Note,Progress Note:\n Today there was a meeting w...


In [20]:
query_diagnoses = """
SELECT subject_id, hadm_id, icd9_code
FROM `physionet-data.mimiciii_clinical.diagnoses_icd`
"""
query_job = bq_client.query(query_diagnoses)
df_diagnoses_icd = query_job.to_dataframe()

# Display the first few rows
df_diagnoses_icd.head()

,subject_id,hadm_id,icd9_code
0,256,108811,53240
1,256,108811,41071
2,256,108811,53560
3,256,108811,40390
4,256,108811,5859


In [21]:
# Keep only Discharge Summary notes and create a merge key
dischargetext_df = df_noteevents.loc[df_noteevents['category'] == 'Discharge summary',
                                     ['subject_id', 'hadm_id', 'text']]

# Create a combined key for merging
dischargetext_df['subj_hadm'] = list(zip(dischargetext_df['subject_id'].astype(int),
                                         dischargetext_df['hadm_id'].astype(int)))

# Display the first few rows
dischargetext_df.head()

,subject_id,hadm_id,text,subj_hadm
217,51257,145513,Admission Date: [**2187-4-27**] ...,"(51257, 145513)"
218,15428,100701,Admission Date: [**2160-4-27**] ...,"(15428, 100701)"
219,13033,133210,Admission Date: [**2186-2-25**] ...,"(13033, 133210)"
220,781,163526,Admission Date: [**2117-9-21**] ...,"(781, 163526)"
221,5035,101276,Admission Date: [**2120-10-4**] ...,"(5035, 101276)"


In [33]:
malignant_mast_cell_icd9 = [
    "20260", "20261", "20262", "20263", "20264",
    "20265", "20266", "20267", "20268"
]

In [34]:
query_disease = f"""
SELECT subject_id, hadm_id, icd9_code
FROM `physionet-data.mimiciii_clinical.diagnoses_icd`
WHERE icd9_code IN ({', '.join(f"'{code}'" for code in malignant_mast_cell_icd9)})
"""
query_job = bq_client.query(query_disease)
df_disease = query_job.to_dataframe()
df_disease.head()

,subject_id,hadm_id,icd9_code
0,3178,131757,20260
1,19308,120012,20260
2,19308,137375,20260
3,19308,180764,20260
4,23707,104817,20260


In [35]:
df_disease['subj_hadm'] = list(zip(df_disease['subject_id'].astype(int),
                                   df_disease['hadm_id'].astype(int)))

In [36]:
# Merge disease subset with discharge summary text
df_disease_notes = df_disease.merge(dischargetext_df, on="subj_hadm", how="inner")
df_disease_notes.head()


,subject_id_x,hadm_id_x,icd9_code,subj_hadm,subject_id_y,hadm_id_y,text
0,3178,131757,20260,"(3178, 131757)",3178,131757,Admission Date: [**2163-4-6**] D...
1,19308,120012,20260,"(19308, 120012)",19308,120012,Admission Date: [**2143-3-28**] ...
2,19308,137375,20260,"(19308, 137375)",19308,137375,Admission Date: [**2140-10-28**] ...
3,19308,180764,20260,"(19308, 180764)",19308,180764,Admission Date: [**2142-7-25**] ...
4,23707,104817,20260,"(23707, 104817)",23707,104817,Admission Date: [**2147-11-8**] Discharge...


In [40]:
df_disease_notes.to_csv("malignanttumor_notes.tsv", sep="\t", index=False)

In [41]:
# Load the saved file to verify
df_check = pd.read_csv("malignanttumor_notes.tsv", sep="\t")
df_check.head()

,subject_id_x,hadm_id_x,icd9_code,subj_hadm,subject_id_y,hadm_id_y,text
0,3178,131757,20260,"(3178, 131757)",3178,131757,Admission Date: [**2163-4-6**] D...
1,19308,120012,20260,"(19308, 120012)",19308,120012,Admission Date: [**2143-3-28**] ...
2,19308,137375,20260,"(19308, 137375)",19308,137375,Admission Date: [**2140-10-28**] ...
3,19308,180764,20260,"(19308, 180764)",19308,180764,Admission Date: [**2142-7-25**] ...
4,23707,104817,20260,"(23707, 104817)",23707,104817,Admission Date: [**2147-11-8**] Discharge...


In [42]:
from google.colab import files
files.download("malignanttumor_notes.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>